In [1]:
from IPython.display import display, HTML, Image
display(HTML("<style>.container { width:85% !important; }</style>"))
%config IPCompleter.use_jedi=False

In [3]:
from bert_score import score
import torch
from transformers import AutoTokenizer, AutoModel
import torch

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [18]:
nl_tokens=tokenizer.tokenize("return maximum value")
code_tokens=tokenizer.tokenize("def max(a,b): if a>b: return a else return b")

In [19]:
tokens=[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]+code_tokens+[tokenizer.sep_token]

In [25]:
tokenizer("def max(a,b): if a>b: return a else return b", return_tensors='pt')

{'input_ids': tensor([[    0,  9232, 19220,  1640,   102,     6,   428,  3256,   114,    10,
         15698,   428,    35,   671,    10,  1493,   671,   741,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [118]:
with torch.no_grad():
    o = model(**tokenizer(["def max(a,b): if a>b: return a else return b", "def min(i,j): if i>j: return i else return j"], return_tensors='pt'))

In [63]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

In [119]:
o.pooler_output.shape

torch.Size([2, 768])

In [80]:
o.pooler_output[0,:].unsqueeze(0).shape

torch.Size([1, 768])

In [82]:
import pandas as pd

In [84]:
df = pd.read_json('../code_datasets/code_crawl.jsonl', lines=True)

In [122]:
df[df['language_name'] == 'Python'].sample()['code']

43448    import sys\nimport pygame\n \npygame.init()\n ...
Name: code, dtype: object

In [129]:
df[df['language_name'] == 'Python']['code'].apply(len).describe()

count     1136.000000
mean      1081.649648
std       1916.171263
min          3.000000
25%        245.750000
50%        640.500000
75%       1300.250000
max      46867.000000
Name: code, dtype: float64

In [107]:
print(c)

>>> def duration(seconds):
	t= []
	for dm in (60, 60, 24, 7):
		seconds, m = divmod(seconds, dm)
		t.append(m)
	t.append(seconds)
	return ', '.join('%d %s' % (num, unit)
			 for num, unit in zip(t[::-1], 'wk d hr min sec'.split())
			 if num)
 
>>> for seconds in [7259, 86400, 6000000]:
	print("%7d sec = %s" % (seconds, duration(seconds)))
 
 
   7259 sec = 2 hr, 59 sec
  86400 sec = 1 d
6000000 sec = 9 wk, 6 d, 10 hr, 40 min
>>> 


In [120]:
cos(o.pooler_output[0,:].unsqueeze(0), o.pooler_output[1,:].unsqueeze(0))

tensor([0.9992])

In [30]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def max(i,j): if i>j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

CPU times: user 1.33 s, sys: 328 ms, total: 1.66 s
Wall time: 5.72 s


(tensor([0.8903]), tensor([0.8903]), tensor([0.8903]))

In [32]:
%%time
Precision, Recall, F1 = score(["def max(a,b): if a>b: return a else return b"], ["def min(i,j): if i<j: return i else return j"], 
                 model_type="microsoft/graphcodebert-base", num_layers=12)
Precision, Recall, F1

Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

CPU times: user 1.28 s, sys: 186 ms, total: 1.47 s
Wall time: 5.55 s


(tensor([0.8669]), tensor([0.8669]), tensor([0.8669]))

In [26]:
print(f"{Precision.numpy()=}, {Recall=}, {F1=}")

Precision.numpy()=array([0.8903439], dtype=float32), Recall=tensor([0.8903]), F1=tensor([0.8903])


In [157]:
Precision.numpy()

array([0.9680341], dtype=float32)

In [14]:
import datasets

In [16]:
from datasets import load_dataset

dataset_py = load_dataset("code_x_glue_cc_code_completion_line", "python")
datasert_j = load_dataset("code_x_glue_cc_code_completion_line", "java")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset code_x_glue_cc_code_completion_line downloaded and prepared to /tf/data/cache/HF/datasets/code_x_glue_cc_code_completion_line/python/0.0.0/4ce8a216b87c5b130aad675f2bbf3612cc1f7fa5dbdc2fcb9c412765cf7830a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset code_x_glue_cc_code_completion_line downloaded and prepared to /tf/data/cache/HF/datasets/code_x_glue_cc_code_completion_line/java/0.0.0/4ce8a216b87c5b130aad675f2bbf3612cc1f7fa5dbdc2fcb9c412765cf7830a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
bloom = AutoModel.from_pretrained('bigscience/bloom-350m')

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

In [24]:
print(dataset_py['train'][0]['input'].replace('<EOL>', '\n'))

<s> from __future__ import absolute_import 
 import weakref 
 import operator 
 from . compat import threading , itertools_filterfalse 
 from . import py2k 
 import types 
 EMPTY_SET = frozenset ( ) 
 class KeyedTuple ( tuple ) : 
 def __new__ ( cls , vals , labels = None ) : 
 t = tuple . __new__ ( cls , vals ) 
 t . _labels = [ ] 
 if labels : 
 t . __dict__ . update ( zip ( labels , vals ) ) 
 t . _labels = labels 
 return t 
 def keys ( self ) : 
 return [ l for l in self . _labels if l is not None ] 
 @ property 
 def _fields ( self ) : 
 return tuple ( self . keys ( ) ) 
 def _asdict ( self ) : 
 return dict ( ( key , self . __dict__ [ key ] ) for key in self . keys ( ) ) 
 class ImmutableContainer ( object ) : 
 def _immutable ( self , * arg , ** kw ) : 
 raise TypeError ( "" % self . __class__ . __name__ ) 
 __delitem__ = __setitem__ = __setattr__ = _immutable 
 class immutabledict ( ImmutableContainer , dict ) : 
 clear = pop = popitem = setdefault = update = ImmutableContaine